#  對於資料初步了解跟遺失值處理

In [1]:
import pandas as pd
import numpy as np
original_data=pd.read_csv("IMDB-Movie-Data.csv")

In [2]:
original_data.head()

,Rank,Title,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,55,The Dark Knight,Action|Crime|Drama,Christopher Nolan,Christian Bale| Heath Ledger| Aaron Eckhart|Mi...,2008,152,9.0,1791916,533.32,82.0
1,81,Inception,Action|Adventure|Sci-Fi,Christopher Nolan,Leonardo DiCaprio| Joseph Gordon-Levitt| Ellen...,2010,148,8.8,1583625,292.57,74.0
2,118,Dangal,Action|Biography|Drama,Nitesh Tiwari,Aamir Khan| Sakshi Tanwar| Fatima Sana Shaikh|...,2016,161,8.8,48969,11.15,NaN
3,37,Interstellar,Adventure|Drama|Sci-Fi,Christopher Nolan,Matthew McConaughey| Anne Hathaway| Jessica Ch...,2014,169,8.6,1047747,187.99,74.0
4,97,Kimi no na wa,Animation|Drama|Fantasy,Makoto Shinkai,Ryunosuke Kamiki| Mone Kamishiraishi| Ryo Nari...,2016,106,8.6,34110,4.68,79.0


In [3]:
original_data.isna().sum()
#處理遺失值
mask1=original_data['Revenue (Millions)'].isna()
mask2=original_data["Metascore"].isna()
original_data["Revenue (Millions)"][mask1]=0
original_data["Metascore"][mask2]=0

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
Rank                  1000 non-null int64
Title                 1000 non-null object
Genre                 1000 non-null object
Director              1000 non-null object
Actors                1000 non-null object
Year                  1000 non-null int64
Runtime (Minutes)     1000 non-null int64
Rating                1000 non-null float64
Votes                 1000 non-null int64
Revenue (Millions)    1000 non-null float64
Metascore             1000 non-null float64
dtypes: float64(3), int64(4), object(4)
memory usage: 86.0+ KB


###  (1)Top‐3 movies with the highest ratings in 2016? 

In [13]:
select_col_data=original_data[["Title","Year","Rating"]]
year_2016_data=select_col_data[select_col_data["Year"]==2016]
year_2016_data.sort_index(by = "Rating",ascending=False)[0:3].reset_index(drop=True)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


,Title,Year,Rating
0,Dangal,2016,8.8
1,Kimi no na wa,2016,8.6
2,Koe no katachi,2016,8.4


###  (2)The actor generating the highest average revenue?

In [6]:
select_col_data=original_data[["Actors","Revenue (Millions)"]]
actor_name_in_each_movie=[]
actor_revenue_in_each_movie=[]
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=names.split("|")
    for one_actor in actors:
        one_actor=one_actor.strip()
        actor_name_in_each_movie.append(one_actor)
        actor_revenue_in_each_movie.append(float(select_col_data["Revenue (Millions)"][x]))
actor_revenue_list=pd.DataFrame({"name":actor_name_in_each_movie,"revenue":actor_revenue_in_each_movie})
unique_actor_list=actor_revenue_list.groupby("name").mean()
max_avg_revenue_actor=unique_actor_list.sort_index(by="revenue",ascending=False)
max_avg_revenue_actor.head()

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  del sys.path[0]


,revenue
name,
John Boyega,936.63
Daisy Ridley,936.63
Donnie Yen,532.17
Alan Tudyk,532.17
Ellen DeGeneres,486.29


###  (3)The average rating of Emma Watson’s movies? 

In [7]:
select_col_data=original_data[["Actors","Rating"]]
actor_name_in_each_movie=[]
actor_rating_in_each_movie=[]
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=names.split("|")
    for one_actor in actors:
        one_actor=one_actor.strip()
        actor_name_in_each_movie.append(one_actor)
        actor_rating_in_each_movie.append(float(select_col_data["Rating"][x]))
actor_rating_list=pd.DataFrame({"name":actor_name_in_each_movie,"Rating":actor_rating_in_each_movie})
unique_actor_list=actor_rating_list.groupby("name").mean()
unique_actor_list[unique_actor_list.index=="Emma Watson"]

,Rating
name,
Emma Watson,7.175


###  (4)Top‐3 directors who collaborate with the most actors? 

In [8]:
select_col_data=original_data[["Director","Actors"]]
director_in_each_movie=[]
actor_name_in_each_movie=[]
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=names.split("|")
    for one_actor in actors:
        one_actor=one_actor.strip()
        actor_name_in_each_movie.append(one_actor)
        director_in_each_movie.append(select_col_data["Director"][x])
director_co_actor_list=pd.DataFrame({"Director":director_in_each_movie,"name":actor_name_in_each_movie})
count_director_co_actors=director_co_actor_list.groupby("Director").apply(lambda x: len(np.unique(x))-1)
count_director_co_actors=count_director_co_actors.reset_index().rename(columns ={0:"count"})
count_director_co_actors.sort_index(by="count",ascending=False).head(3)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  


,Director,count
518,Ridley Scott,28
391,M. Night Shyamalan,24
488,Paul W.S. Anderson,20


###  (5)Top‐2 actors playing in the most genres of movies? 

In [9]:
select_col_data=original_data[["Actors","Genre"]]
genre_in_each_movie=[]
actor_name_in_each_movie=[]
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=names.split("|")
    genre=[genre.strip() for genre in select_col_data["Genre"][x].split("|")]
    for one_actor in actors:
        for one_genre in genre:
            one_actor=one_actor.strip()
            actor_name_in_each_movie.append(one_actor)
            genre_in_each_movie.append(one_genre)
actor_genres_list=pd.DataFrame({"name":actor_name_in_each_movie,"genre":genre_in_each_movie})
actor_genres_list=actor_genres_list.groupby("name").apply(lambda x: len(np.unique(x))-1)
actor_genres_list=actor_genres_list.reset_index().rename(columns ={0:"count"})
actor_genres_list.sort_index(by="count",ascending=False).head(6)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  app.launch_new_instance()


,name,count
256,Brad Pitt,14
355,Chloe Grace Moretz,13
956,Johnny Depp,13
97,Amy Adams,13
1715,Scarlett Johansson,13
741,Hugh Jackman,13


###  (6)Top‐3 actors whose movies lead to the largest maximum gap of years?

In [10]:
select_col_data=original_data[["Actors","Year"]]
year_in_each_movie=[]
actor_name_in_each_movie=[]
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=names.split("|")
    for one_actor in actors:
            one_actor=one_actor.strip()
            actor_name_in_each_movie.append(one_actor)
            year_in_each_movie.append(select_col_data["Year"][x])
actor_years_list=pd.DataFrame({"name":actor_name_in_each_movie,"years_diff":year_in_each_movie})
actor_years_list=actor_years_list.groupby("name").apply(lambda x: max(np.unique(x))-min(np.unique(x)))
actor_years_list=actor_years_list.reset_index().rename(columns ={0:"years_diff"})
actor_years_list=actor_years_list.sort_index(by="years_diff",ascending=False)
actor_years_list[actor_years_list["years_diff"]==10]

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  


,name,years_diff
495,Denzel Washington,10
1669,Sacha Baron Cohen,10
299,Bryce Dallas Howard,10
591,Emily Blunt,10
895,Jessica Biel,10
1025,Justin Theroux,10
1657,Russell Crowe,10
999,Judi Dench,10
741,Hugh Jackman,10
1374,Michelle Monaghan,10


###  (7)Find all actors who collaborate with Johnny Depp in direct and indirect ways 

In [11]:
select_col_data=original_data[["Actors"]]
actor_name_in_each_movie=[]
join_co_actors=[]

#取出每一次電影的演員以及票房
for x in range(1,len(select_col_data)):
    names=select_col_data["Actors"][x]
    actors=[actor_in_movie.strip() for actor_in_movie in names.split("|")]
    for y in range(0,len(actors)):
        for z in range(0,len(actors)):
            if y==z:
                continue
            actor_name_in_each_movie.append(actors[y])
            join_co_actors.append(actors[z])
actor_join_list=pd.DataFrame({"name":actor_name_in_each_movie,"join_co_actors":join_co_actors})
actor_join_list=actor_join_list.groupby("name").apply(lambda x: np.unique(x))

john_co_list=list(actor_join_list[actor_join_list.index=="Johnny Depp"][0])
n=0
while n<len(john_co_list):
    next_list=list(actor_join_list[actor_join_list.index==john_co_list[n]][0])
    new_actors=list(set(next_list).difference(set(john_co_list)))
    john_co_list=john_co_list+new_actors
    n=n+1
john_co_list.remove("Johnny Depp")
print("Find all actors who collaborate with Johnny Depplen:",len(john_co_list))

Find all actors who collaborate with Johnny Depplen: 1574
